In [6]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from numpy import ones,vstack,arctan
from numpy.linalg import lstsq
from math import sqrt,dist
import SpaceDetector as det

In [2]:
nusc = NuScenes(version = 'v1.0-trainval01_blobs', dataroot=r'C:\Users\Stevens User\bdd100k\bdd100k\data\sets\nuscenes')

Loading NuScenes tables for version v1.0-trainval01_blobs...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 40.785 seconds.
Reverse indexing ...
Done reverse indexing in 9.1 seconds.


In [4]:
scenes = []

for i in range(0, 60):
    if nusc.log[i]['location'] == 'boston-seaport' and 'park' in nusc.scene[i]['description']:
        print(nusc.scene[i]['description'])
        scenes.append(nusc.scene[i])

Turn left, pass trucks on both sides, construction workers, sleeping construction workers, garbage, parked cars
Turn left, people on both sides, parked vans
Turn right, walking peds, pedestrian in blind spot in couple of frames, car park exit
Overtake parked car, bus turning, motorcycle
Taxi parked on road, bicycle rack, many peds, many jaywalkers, parked bicycles, ped exiting taxi, ped crossing traffic light, lane change, peds crossing crosswalk
No traffic, dead end street, overtake parked truck
Busy street, parked cars, parking lot, ped sitting at bus stop, motorcycle driving
Parked cars, many peds, parked bicycles, trash can, parking lots
Truck turns, parked cars, bump, bikes
Full parking lot, peds, exiting from car park
Construction, barriers behind net, parked trucks, excavators, turn left
Narrow street, seveal peds, shadows, turn left, parked motorcycle
Pass parked vehicles, peds, arrive at intersection
Oncoming car in the same lane, overtake parked car, turn left
Peds, nature, c

In [7]:
for scene in scenes:
    first_token = scene['first_sample_token']
    last_token = scene['last_sample_token']
    nbr_samples = scene['nbr_samples']
    current_token = first_token
    
    parkedCars = []
    i = 0
    while current_token != last_token:
        sample = nusc.get('sample', current_token)
        if i % 10 == 0:
            #print('sample ' + str(i))
            for ann in sample['anns']:
                car = nusc.get('sample_annotation', ann)
                if len(car['attribute_tokens']) > 0:
                    attr = nusc.get('attribute', car['attribute_tokens'][0])['name']
                    if attr == 'vehicle.parked':
                        parkedCars.append([car['rotation'][0], ann])
            
            if len(parkedCars) > 3:
                parkedCars.sort(key = lambda x: abs(x[0]))
                #print(parkedCars)
                first_car = parkedCars[0][1]
                xL = nusc.get('sample_annotation', first_car)['translation'][0]
                xR = xL
                yL = nusc.get('sample_annotation', parkedCars[0][1])['translation'][1]
                yR = yL
                #find lots
                k = 0
                for j in range(len(parkedCars) - 1):
                    this_car = nusc.get('sample_annotation', parkedCars[j][1])
                    next_car = nusc.get('sample_annotation', parkedCars[j+1][1])
                    #print(abs(abs(this_car['rotation'][0]) - abs(next_car['rotation'][0])))
                    if abs(abs(this_car['rotation'][0]) - abs(next_car['rotation'][0])) < 0.03:
                        if this_car['translation'][0] < xL:
                            xL = this_car['translation'][0]
                            yL = this_car['translation'][1]
                        elif this_car['translation'][0] > xR:
                            xR = this_car['translation'][0]
                            yR = this_car['translation'][1]
                        k += 1
                    else:
                        if k > 2:
                            lot = [xL,yL,xR,yR]
                            print(lot)
                            spaces = det.findSpace(nusc,sample,xL,yL,xR,yR,0)
                            print(spaces)
                        xL = this_car['translation'][0]
                        yL = this_car['translation'][1]
                        xR = xL
                        yR = yL
                        k = 0
                        
                        
                
                
                
        current_token = sample['next']
        i += 1

[323.109, 996.664, 344.221, 1046.518]
Line Solution is y = 2.36140583554376x + 233.6725218832911
[[4.177, 324.758, 1000.912], [5.134, 327.235, 1007.076], [5.403, 330.188, 1013.597], [6.616, 333.338, 1020.844], [11.019, 337.278, 1030.87], [9.627, 341.993, 1041.689]]
[282.4, 972.575, 344.221, 1046.518]
Line Solution is y = 1.1960822374274167x + 634.8013761504969
[[2.563, 283.062, 971.283], [97.212, 313.972, 1008.254]]
[249.933, 924.113, 344.221, 1046.518]
Line Solution is y = 1.2982033768878347x + 599.6491354042928
[[57.189, 266.447, 948.009], [95.636, 313.591, 1009.211]]
[665.941, 773.28, 671.436, 769.632]
Line Solution is y = -0.6638762511373857x + 1215.382414558682


IndexError: list index out of range

ModuleNotFoundError: No module named 'GarbageCollect'